#colab 사용을 위한 사전 작업

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/analysis_application')
print(sys.path)

In [ ]:
!pip install sktime

#import
 tracknet: test 폴더 안에 있는 코드

In [ ]:
import cv2
import argparse
import queue
import numpy
import pandas
import pickle
import time
import trackplayers
from collections import deque
import pandas as pd
import numpy as np
from copy import deepcopy

from sktime.datatypes._panel._convert import from_2d_array_to_nested
from PIL import Image, ImageDraw
from pickle import load
from tracknet import TrackNet
from bounce import *

# 변수 정의 및 사전 작업

In [ ]:
from numpy.core.numeric import rollaxis
# 변수정의
current_frame = 0
tracknet_width, tracknet_height = 640, 360
bounce = 1
coords = []
check_time = []
frames = []
frame, frame1, frame2 = None, None, None
n_classes=256


# 궤도를 그리기위한 프레임 7장 저장
trajectory_ball = deque()
for i in range(0, 8):
    trajectory_ball.appendleft(None)

path = '/content/drive/MyDrive/analysis_application'
input_video_path = path + '/video/video_cut.mp4'
# input_video_path = path + '/video/newvideo_cut.mp4'
# input_video_path = path + '/video/input_video.mp4'
output_video_path = path + '/video/video_output.mp4'
model_path = path + '/Weight_ball/model.3'
bounce_clf_path = path + '/Weight_ball/clf.pkl'
yolo_label_path = path + '/Yolov3/yolov3.txt'
yolo_weight_path = path + '/Yolov3/yolov3.weights'
yolo_cfg_path = path + '/Yolov3/yolov3.cfg'
tracking_players_path = path + '/tracking_players.csv'

# 영상불러오기 및 영상정보 추출
video = cv2.VideoCapture(input_video_path)
fps = int(video.get(cv2.CAP_PROP_FPS))
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# 영상정보 출력
print('fps : {}'.format(fps))
print('frame sizee : {}x{}'.format(frame_width, frame_height))
print('num_frames :{}'.format(num_frames))

# 트렉넷 모델 불러오기 및 컴파일
modelFN = TrackNet
model = modelFN(n_classes, tracknet_height, tracknet_width)
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta', metrics=['accuracy'])

model.load_weights(model_path)

In [ ]:
# 영상 저장을 위한 셋팅
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(
    output_video_path, fourcc, fps, (frame_width, frame_height))

# 선수트래킹


In [ ]:
# yolov3
# 라벨링한다 -> 변수정의라 생각함
LABELS = open(yolo_label_path).read().strip().split("\n")
# 네트워크 불러오기 -> opencv로 딥러닝을 실행하기 위해 생성
net = cv2.dnn.readNet(yolo_weight_path, yolo_cfg_path)

# 플레이어 트래커
ct_players = trackplayers.CentroidTracker()
# 선수 위치 저장할 변수
players_positions = {'x_0': [], 'y_0': [], 'x_1': [], 'y_1': []}

while True:
    ret, frame = video.read()
    current_frame += 1
    print('tracking the players : {}%, number of frames : {}'.format(
        round((current_frame/num_frames)*100), current_frame))
    if not ret:
      break
    # 프레임 사이즈 및 타입 수정을 위한 복사

    output_frame = frame

    scale = 0.00392
    # 입력 영상을 blob 객체로 만들기 -> 해당 인자들은 모델 파일의 학습에 맞게 입력되어있음
    # blob: Binary Large Object 의 약자. 즉, 바이너리 형태로 큰 객체(이미지, 사운드 등)를 저장
    blob = cv2.dnn.blobFromImage(
        frame, scale, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)  # 네트워크 입력 설정
    # 네트워크 순방향 실행을 위한 코드
    # input: 출력 레이어 이름 리스트
    # output: 지정한 레이어의 출력 블롭 리스트
    outs = net.forward(trackplayers.get_output_layer(net))
    # 선수들 위치
    detected_players = trackplayers.predict_players(outs, LABELS, frame, 0.8)
    print(detected_players)


    # map 함수는 첫번째 매개변수에 함수, 두번째 매개변수에 반복 가능한 자료형(리스트, 튜플 등)
    # map 함수의 반환 값은 map 객체-> 해당 자료형을 list 혹은 tuple 로 형변환 필요
    # map(적용시킬 함수, 적용할 값들)
    # track players with a unique ID
    format_detected_players = list(map(trackplayers.update_boxes, list(detected_players)))
    players_objects = ct_players.update(format_detected_players)

    # players positions frame by frame
    print(players_positions)
    print(format_detected_players)
    print(players_objects)

    try:
      players_positions['x_0'].append(tuple(players_objects[0])[0])
      players_positions['y_0'].append(tuple(players_objects[0])[1])
      players_positions['x_1'].append(tuple(players_objects[1])[0])
      players_positions['y_1'].append(tuple(players_objects[1])[1])
    except KeyError:
      print("KeyError")



    players_positions['x_0'].append(tuple(players_objects[0])[0])
    players_positions['y_0'].append(tuple(players_objects[0])[1])
    players_positions['x_1'].append(tuple(players_objects[1])[0])
    players_positions['y_1'].append(tuple(players_objects[1])[1])

    # draw players boxes
    color_box = (0, 0, 255)

    # 선수 draw
    if len(detected_players) > 0:
        for box in detected_players:
            print(box)
            x, y, w, h = box
            cv2.rectangle(output_frame, (x, y), (x + w, y + h), color_box, 2)

    # draw tracking id of each player
    for (objectID, centroid_player) in players_objects.items():
        # draw both the ID of the object and the centroid of the
        # object on the output frame
        text = "ID {}".format(objectID)
        cv2.putText(output_frame, text, (centroid_player[0] - 50, centroid_player[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        cv2.circle(
            output_frame, (centroid_player[0], centroid_player[1]), 1, (0, 255, 0), 2)
    frames.append(output_frame)
video.release
output_video.release
current_frame = 0
print("complete tracking the players")

# 공트래킹

In [ ]:
video.set(1,current_frame);
ret, frame1 = video.read()

output_video.write(frame1)
current_frame += 1

# 프레임 사이즈 및 타임 수정
frame1 = cv2.resize(frame1, (tracknet_width, tracknet_height))
frame1 = frame1.astype(numpy.float32)


video.set(1,current_frame);
ret, frame = video.read()

output_video.write(frame)
current_frame += 1

# 프레임 사이즈 및 타임 수정
frame = cv2.resize(frame, (tracknet_width, tracknet_height))
frame = frame.astype(numpy.float32)

last = time.time()  # start counting

# 프레임단위로 반복
while(True):
    current_frame += 1
    print('percentage of video processed : {}%'.format(round((current_frame/num_frames)*100)))

    # 프레임 사이즈 및 타입 수정을 위한 복사
    frame2 = frame1
    frame1 = frame

    video.set(1,current_frame);
    ret, frame = video.read()

    if not ret:
      break

    output_frame = frame

    # 프레임 사이즈 및 타임 수정
    frame = cv2.resize(frame, (tracknet_width, tracknet_height))
    frame = frame.astype(numpy.float32)

    # 3개의 프레임을 합침
    X = numpy.concatenate((frame,frame1,frame2),axis=2)
    X = numpy.rollaxis(X, 2, 0)

    # 히트맵 예측
    predict = model.predict(numpy.array([X]))[0]
    predict = predict.reshape(
        (tracknet_height, tracknet_width, n_classes)).argmax(axis=2)

    # cv2이미지 uint8로 변경해야만함
    predict = predict.astype(numpy.uint8)
    heatmap = cv2.resize(predict, (frame_width, frame_height))

    # 이미지 이진화작업 및 공 후보 트래킹 출력
    ret, heatmap = cv2.threshold(heatmap, 127, 255, cv2.THRESH_BINARY)
    circles = cv2.HoughCircles(heatmap, cv2.HOUGH_GRADIENT, dp=1,
                               minDist=1, param1=50, param2=2, minRadius=2, maxRadius=7)
    PIL_image = cv2.cvtColor(output_frame, cv2.COLOR_BGR2RGB)
    PIL_image = Image.fromarray(PIL_image)

    # 공후보 트래킹 성공 시 트래킹 표시
    if circles is not None:
        # 공후보가 하나 일 시 트래킹 표시
        if len(circles) == 1:
            print("공후보 하나")
            x = int(circles[0][0][0])
            y = int(circles[0][0][1])

            coords.append([x, y])
            check_time.append(time.time()-last)
            trajectory_ball.appendleft([x, y])
            trajectory_ball.pop()

        # 두 개 이상일 시 트래킹 표시하지 않음
        else:
            print("공후보 두개 이상")
            coords.append(None)
            check_time.append(time.time()-last)
            trajectory_ball.appendleft(None)
            trajectory_ball.pop()

    # 공 후보 트래킹 실패 시 트래킹 표시하지 않음
    else:
        coords.append(None)
        check_time.append(time.time()-last)
        trajectory_ball.appendleft(None)
        trajectory_ball.pop()


    # 전 7장의 프레임 후보공 draw
    for i in range(0, 8):
        if trajectory_ball[i] is not None:
            draw_x = trajectory_ball[i][0]
            draw_y = trajectory_ball[i][1]
            position_circle = (draw_x - 2, draw_y-2, draw_x+2, draw_y+2)
            draw = ImageDraw.Draw(PIL_image)
            draw.ellipse(position_circle, outline='yellow')
            del draw

    opencvImage = cv2.cvtColor(numpy.array(PIL_image), cv2.COLOR_RGB2BGR)
    output_video.write(opencvImage)

video.release()
output_video.release()

input_video 3분 43초

# 선수 위치 저장

In [ ]:
# players positions
df_players_positions = pd.DataFrame()
df_players_positions['x_0'] = players_positions['x_0']
df_players_positions['y_0'] = players_positions['y_0']
df_players_positions['x_1'] = players_positions['x_1']
df_players_positions['y_1'] = players_positions['y_1']
df_players_positions.to_csv("tracking_players.csv")

# 공 위치 저장 (csv)

In [ ]:
import csv 
print(check_time)

# coords 내용 -> 다시 돌리기 싫으니깐 이거 복사해서 사용
coords = [[1059, 91], None, None, None, None, None, None, None, None, None, None, None, [987, 328], [978, 356], [972, 382], None, None, [950, 455], [580, 304], [931, 525], [921, 561], [912, 597], [912, 596], [901, 634], [896, 647], [886, 650], [1192, 306], [873, 660], [872, 661], [865, 665], [855, 672], [846, 681], [840, 690], [830, 698], [831, 699], [822, 708], [814, 718], [805, 732], [799, 745], [789, 757], [792, 756], [780, 769], [784, 750], [788, 722], [795, 697], [801, 677], [800, 680], [805, 655], [810, 635], [817, 613], [822, 595], [825, 576], [824, 579], [829, 561], [834, 544], [838, 529], [841, 516], [846, 501], [846, 501], [850, 487], [853, 475], [856, 462], [859, 451], [862, 438], [862, 438], [867, 427], [868, 418], [873, 409], [876, 402], [878, 393], [878, 392], [880, 385], [883, 379], [886, 370], [889, 363], [890, 358], [891, 358], [892, 354], [895, 348], [898, 342], [900, 337], [901, 333], [903, 334], [904, 331], [906, 327], [908, 322], [910, 321], [913, 316], [913, 316], [913, 313], [916, 313], [918, 310], [919, 310], [923, 309], [923, 309], [924, 309], [927, 309], [927, 309], [929, 309], [930, 309], [931, 307], [933, 309], [934, 309], [936, 309], [936, 310], [938, 311], [937, 312], [939, 312], [939, 313], [939, 315], [940, 316], [942, 321], [942, 320], [943, 324], [946, 326], [948, 325], [948, 316], [948, 309], [948, 309], [948, 300], [949, 292], [949, 286], [951, 280], [951, 274], [952, 274], [951, 268], [952, 262], [952, 254], [954, 250], [954, 246], [952, 247], [954, 243], [954, 240], [950, 238], [946, 238], [940, 241], [941, 243], [936, 243], [930, 246], [925, 251], [919, 254], [914, 258], [915, 259], [908, 263], [903, 270], [897, 276], [892, 280], [886, 287], [886, 288], [880, 293], [877, 301], [870, 312], [864, 319], [858, 330], [859, 331], [853, 339], [849, 351], [843, 360], [837, 373], [831, 386], [831, 385], [826, 398], [819, 407], [814, 428], [829, 394], [803, 453], [805, 454], [799, 469], [793, 486], [787, 504], [781, 520], [775, 537], [774, 538], [769, 555], [763, 575], [758, 593], [754, 615], [747, 634], [748, 633], [739, 655], [733, 676], [730, 697], [721, 724], [718, 733], [720, 734], [714, 732], [711, 732], [706, 730], [702, 729], [697, 729], [697, 728], [694, 729], [690, 730], [685, 730], [682, 733], [678, 736], [676, 736], [672, 739], [667, 742], [664, 746], [661, 751], [657, 756], [655, 756], [652, 760], [646, 767], [643, 774], [639, 781], [636, 788], [634, 790], [630, 796], [627, 807], [632, 773], [633, 742], [637, 715], [636, 718], [641, 683], [645, 660], [648, 636], [652, 615], [655, 594], [655, 595], [658, 572], [660, 556], [660, 538], [664, 524], [666, 510], [666, 511], [667, 494], [666, 796], [672, 469], [672, 455], [674, 448], [675, 448], [673, 436], [676, 429], [678, 418], [678, 411], [682, 403], [682, 403], [681, 394], [680, 389], [681, 385], [684, 381], [683, 375], [685, 376], [685, 372], [685, 369], [685, 364], [685, 363], [687, 360], [688, 360], [688, 357], [688, 357], [688, 356], [689, 355], [870, 209], [691, 354], [689, 352], [688, 340], [688, 330], [690, 318], [690, 310], [691, 311], [690, 301], [690, 292], [690, 285], [690, 276], [691, 268], [1198, 304], [691, 264], [690, 256], [690, 251], [691, 246], [691, 241], [692, 240], [690, 235], [690, 231], [691, 226], [715, 220], [701, 227], [700, 225], [709, 234], [713, 237], [715, 229], None, None, None, None, None, None, None, None, None, [765, 240], [771, 241], [776, 244], [783, 249], [790, 250], [790, 251], [795, 255], [803, 257], [809, 262], [815, 264], [820, 268], [819, 268], [828, 274], [834, 277], [843, 280], [849, 286], [855, 291], [855, 291], [862, 294], [868, 301], [877, 309], [882, 313], [888, 318], [889, 319], [895, 325], [903, 331], [910, 338], [610, 211], [925, 354], [925, 354], [933, 361], [937, 367], [949, 379], [953, 386], [961, 393], None, [969, 403], [976, 411], [982, 421], [991, 430], [998, 441], [999, 441], [1009, 454], [1013, 461], [1022, 474], [1200, 307], [1037, 494], [1038, 496], [1046, 509], [1054, 522], [1060, 534], [1070, 545], [1080, 561], [1080, 560], [1087, 573], [1096, 587], [1105, 603], [1113, 616], [1124, 632], [1121, 629], [1129, 643], [1137, 662], None, [1155, 688], None, None, None, None, None, None, None, None, None, [1210, 721], [1216, 721], [1223, 719], [1228, 720], [1227, 718], [1234, 718], [1241, 722], None, [1252, 721], [1257, 723], [1258, 723], [1264, 727], [1270, 729], [1275, 732], [1282, 735], [1288, 739], [1288, 739], [1294, 742], [1300, 747], [1307, 752], [1307, 729], [1301, 692], [1301, 692], [1299, 667], [1292, 639], [1289, 611], [1285, 591], [1281, 565], [1281, 569], [1278, 547], [1276, 529], [1272, 512], [1265, 491], [1264, 478], None, [1262, 463], [1259, 449], [1255, 439], [1252, 429], [1249, 415], [1249, 418], [1246, 409], [1243, 400], [1242, 391], [1238, 384], [1236, 380], [1237, 381], [1233, 374], [1230, 367], [1226, 363], [1225, 361], [1222, 360], [1224, 360], [1220, 358], [1218, 354], [1216, 354], [1213, 352], [1213, 352], [1213, 352], [1210, 352], [1207, 352], [1204, 352], [1202, 347], [1203, 334], [1202, 338], [1201, 325], [1199, 312], [1199, 299], [1198, 292], [1197, 283], [1197, 283], [1195, 273], [1194, 265], [1192, 256], [1192, 248], [1190, 241], [1190, 242], [1192, 234], [1189, 229], [1188, 222], [1187, 215], [1186, 211], [1189, 212], [1185, 207], [1185, 202], [1183, 199], [1182, 196], [1183, 193], [1183, 195], [1181, 192], [1182, 188], None, None, [1174, 172], None, None, None, [1168, 148], [1165, 139], [1164, 130], None, [1162, 126], [1158, 114], [1154, 110], [1152, 100], [1152, 91], None, [1149, 84], [1146, 79], [1143, 70], [1141, 64], [1138, 57], None, [1135, 49], [1132, 45], [1131, 39], [1128, 33], [1125, 28], None, [1123, 22], [1118, 17], [1116, 11], [1112, 8], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [619, 305], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [997, 10], None, [994, 14], [993, 19], [988, 25], [983, 33], [979, 40], None, [976, 46], [973, 54], [969, 60], [964, 67], [960, 76], None, [1201, 306], [952, 93], [949, 105], [945, 109], [648, 681], None, [936, 131], None, [930, 151], [622, 304], [922, 175], None, None, [913, 199], [907, 214], [906, 224], [903, 238], None, [817, 543], [894, 264], [889, 277], [885, 294], [882, 306], [882, 305], [877, 321], [874, 334], [870, 352], [865, 367], [860, 386], [862, 385], [858, 396], [854, 413], [850, 414], [844, 412], [840, 412], [840, 412], [838, 414], [833, 415], [832, 415], [829, 418], [827, 419], [826, 420], [822, 420], [820, 421], [816, 424], [813, 428], [810, 420], [811, 420], [807, 399], [806, 374], [803, 355], [801, 338], [799, 319], [799, 322], [799, 310], [796, 290], [793, 274], [792, 261], [790, 247], [792, 250], [789, 237], [786, 224], [784, 213], [782, 203], [780, 192], [780, 193], [781, 182], [777, 174], [774, 166], [772, 160], [768, 152], [771, 153], [769, 148], [768, 142], [766, 136], [765, 132], [762, 128], None, [761, 124], [760, 123], [760, 120], [762, 124], None, None, [771, 139], [775, 150], [781, 159], None, None, None, None, [1183, 305], [811, 214], [816, 226], [821, 239], [821, 239], [829, 253], [832, 264], [837, 276], [843, 291], [850, 309], [851, 307], [854, 321], [862, 338], [865, 352], [869, 368], [877, 387], [876, 388], [881, 405], [885, 420], [891, 438], [877, 391], [900, 475], None, None, None, None, None, None, None, None, None, None, None, None, None, None, [1199, 305], [1200, 307], [1204, 306], [937, 473], [938, 476], [938, 461], [940, 454], [942, 448], [943, 441], [943, 433], [943, 433], [946, 427], [945, 420], [948, 414], [949, 409], [949, 403], [951, 404], [950, 398], [951, 393], [953, 389], [953, 386], [955, 382], [955, 382], [955, 379], [955, 376], [958, 372], [959, 371], [958, 368], [958, 367], [1286, 302], [961, 364], [1281, 303], [961, 363], [964, 364], [964, 364], [964, 363], [964, 364], [964, 364], [966, 367], [967, 369], [968, 368], [967, 370], [967, 372], [969, 373], [970, 375], [972, 378], [972, 379], [970, 382], [970, 384], [970, 388], [972, 390], [973, 396], [973, 396], [973, 399], [975, 405], [974, 408], [974, 413], [975, 421], [975, 421], [974, 425], [975, 431], [975, 436], [976, 445], [975, 451], None, [979, 459], [978, 460], [976, 450], [975, 445], [976, 436], [976, 436], [976, 430], [975, 424], [1199, 306], [976, 412], [975, 408], [975, 408], [975, 402], [975, 397], [975, 395], [974, 389], [1198, 306], [973, 387], [973, 384], [974, 379], [973, 379], [973, 376], [974, 374], [974, 373], None, None, None, None, [973, 364], [974, 365], None, None, None, None, None, None, None, None, [972, 375], None, [971, 377], None, None, [966, 381], None, None, None, None, [971, 397], [970, 403], [971, 407], None, None, None, None, None, [951, 415], [942, 414], [936, 414], [936, 414], [930, 412], [924, 412], [916, 415], [910, 417], [904, 417], [905, 417], [896, 419], [889, 423], [883, 424], [877, 427], [870, 429], [870, 429], [864, 433], [853, 438], [617, 305], [841, 445], [834, 451], [1199, 305], [825, 454], [819, 460], [811, 466], [805, 472], [798, 480], [799, 480], [845, 545], [783, 493], [1036, 450], [768, 511], [760, 517], [762, 517], [753, 525], [745, 535], [739, 544], [733, 554], [731, 548], [732, 549], [737, 530], [739, 516], [745, 502], [703, 583], [753, 474], [753, 477], [757, 459], [760, 448], [1286, 302], [768, 426], [771, 414], [769, 415], [775, 403], [778, 396], [783, 385], [786, 373], [790, 364], [790, 364], [793, 358], [796, 346], [799, 340], [804, 334], [808, 326], [807, 327], [810, 319], [813, 313], [817, 307], [821, 299], [823, 297], [823, 298], [827, 291], [831, 286], [834, 282], [837, 277], [840, 273], [840, 273], [843, 268], [847, 265], [852, 262], [855, 262], [858, 261], [856, 258], [860, 260], [864, 259], None, [870, 258], None, None, None, None, None, [976, 429], [1002, 469], None, None, [1046, 542], [1067, 581], [1083, 597], [1095, 583], [1096, 587], [1108, 571], [1119, 564], [855, 420], [852, 417], [852, 416], [1156, 532], [1168, 523], [1182, 515], [1192, 505], [852, 420], [1221, 491], [1221, 492], [1234, 484], [1248, 477], [1262, 470], [1273, 465], [1282, 463], None, [1302, 457], [1315, 454], [1327, 450], [1344, 445], [1354, 445], [1353, 447], [1367, 441], [1379, 440], None, None, None, None]

# check_time 다시 돌리기 싫으면 이걸로
check_time = [21.219351053237915, 21.9531147480011, 22.60288667678833, 23.4522066116333, 24.078917264938354, 24.722388982772827, 25.585227966308594, 26.215940952301025, 26.86882495880127, 27.74319624900818, 28.39731526374817, 29.281182527542114, 29.968061923980713, 30.65881037712097, 31.341582775115967, 32.020023345947266, 32.68720102310181, 33.399956941604614, 34.30859637260437, 35.011292934417725, 35.73009777069092, 36.46456599235535, 37.171027183532715, 37.92597055435181, 38.66717529296875, 39.40402054786682, 40.17620134353638, 40.91667819023132, 41.69022822380066, 42.45196318626404, 43.44306564331055, 44.21937036514282, 45.00068378448486, 45.76252102851868, 46.53792977333069, 47.34992456436157, 48.12317728996277, 48.91162037849426, 49.69637966156006, 50.48257875442505, 51.29044151306152, 52.315706729888916, 53.12723469734192, 53.942683696746826, 54.76206707954407, 55.631988286972046, 56.47075176239014, 57.528403520584106, 58.59016799926758, 59.4431893825531, 60.275946617126465, 61.1447057723999, 62.05058526992798, 63.1358425617218, 64.2309045791626, 65.33173513412476, 66.21301436424255, 67.31068086624146, 68.19604516029358, 69.08644104003906, 70.0097143650055, 70.90458488464355, 71.81856799125671, 72.93348979949951, 73.84650826454163, 74.77511358261108, 75.69242262840271, 76.60168051719666, 77.51544380187988, 78.67865800857544, 80.22702264785767, 81.18214654922485, 82.34862089157104, 83.5369963645935, 84.51343941688538, 85.46636843681335, 86.44512701034546, 87.40907955169678, 88.38248777389526, 89.37001609802246, 90.34600186347961, 91.581130027771, 92.58260750770569, 93.81060695648193, 95.04333806037903, 96.21791076660156, 97.4504382610321, 98.49916243553162, 99.5215425491333, 100.78487014770508, 101.82790350914001, 103.08344197273254, 104.34798669815063, 105.39505410194397, 106.4581286907196, 107.73388242721558, 109.0205934047699, 110.1168999671936, 111.2093014717102, 112.28735780715942, 113.38998889923096, 114.49104332923889, 115.6369481086731, 116.7750973701477, 118.11721301078796, 119.26323938369751, 120.38030314445496, 121.52405858039856, 122.87288308143616, 124.24742317199707, 125.41536688804626, 126.66100287437439, 128.03630948066711, 129.22006273269653, 130.4364936351776, 131.61194443702698, 132.984717130661, 134.17715764045715, 135.40401315689087, 136.6213481426239, 137.82190585136414, 139.03753209114075, 140.23387694358826, 141.44936990737915, 142.6769609451294, 144.11697363853455, 145.52580332756042, 146.7468740940094, 147.9899742603302, 149.20900058746338, 150.451890707016, 151.6920256614685, 152.93422555923462, 154.17973923683167, 155.64374256134033, 156.92495703697205, 158.4301152229309, 159.6937928199768, 161.18788838386536, 162.68051552772522, 164.20964550971985, 165.73311042785645, 167.03928399085999, 168.3478138446808, 169.67963576316833, 171.0031349658966, 172.34538912773132, 173.67306542396545, 175.02335858345032, 176.35225415229797, 177.73331141471863, 179.10465359687805, 180.670068025589, 182.03324007987976, 183.3822991847992, 184.7292549610138, 186.07948780059814, 187.46278548240662, 188.8353374004364, 190.18704104423523, 191.7546272277832, 193.11090207099915, 194.480815410614, 196.0709729194641, 197.4081392288208, 198.7726185321808, 200.1348922252655, 201.7202067375183, 203.291424036026, 204.64831829071045, 206.02496647834778, 207.44230484962463, 209.09085488319397, 210.72955584526062, 212.3975121974945, 213.81033754348755, 215.4305703639984, 216.84314155578613, 218.49681639671326, 220.17278671264648, 221.66720986366272, 223.11537337303162, 224.60213994979858, 226.0915150642395, 227.57573580741882, 229.02923154830933, 230.47502636909485, 231.93324255943298, 233.5788378715515, 235.25502109527588, 236.75951266288757, 238.24831247329712, 239.74002242088318, 241.2533314228058, 242.80845522880554, 244.5487084388733, 246.06925630569458, 247.57557702064514, 249.052992105484, 250.53830862045288, 252.05493140220642, 253.5780553817749, 255.09527325630188, 256.61754846572876, 258.3957121372223, 259.9736201763153, 261.7761900424957, 263.5803334712982, 265.3808958530426, 266.9904885292053, 268.5697891712189, 270.1461811065674, 271.91901659965515, 273.67332100868225, 275.2228708267212, 276.8238365650177, 278.37811970710754, 279.9230217933655, 281.5277752876282, 283.0937931537628, 284.64566564559937, 286.4393243789673, 288.0789849758148, 289.6844184398651, 291.30883288383484, 292.88063788414, 294.4847047328949, 296.095335483551, 297.925874710083, 299.5573651790619, 301.1810359954834, 303.03322196006775, 304.67188143730164, 306.3060841560364, 307.93092107772827, 309.574099779129, 311.2445511817932, 312.96566438674927, 314.8347237110138, 316.5400059223175, 318.2766647338867, 320.0079011917114, 321.958918094635, 323.919451713562, 325.7000412940979, 327.7080590724945, 329.74761414527893, 331.4992048740387, 333.4658172130585, 335.2859387397766, 337.33847761154175, 339.3174841403961, 341.1052830219269, 342.9444992542267, 344.8086440563202, 346.8504447937012, 348.67012190818787, 350.53238701820374, 352.3903274536133, 354.2333483695984, 356.3057191371918, 358.1815986633301, 360.0650460720062, 360.7555730342865, 361.68425250053406, 362.61142683029175, 363.3302688598633, 364.0516245365143, 364.76418948173523, 365.49658489227295, 366.4437806606293, 367.16630458831787, 367.9013066291809, 368.63680028915405, 369.3702244758606, 370.33069801330566, 371.09755206108093, 371.862820148468, 372.8592793941498, 373.85722494125366, 374.6519913673401, 375.44579672813416, 376.44719433784485, 377.2475075721741, 378.2548072338104, 379.0566282272339, 379.8823974132538, 380.68715262413025, 381.68984031677246, 382.487402677536, 383.2997591495514, 384.122243642807, 385.01868319511414, 386.0499064922333, 387.1048710346222, 387.93663692474365, 388.7769911289215, 389.6140515804291, 390.449196100235, 391.338814496994, 392.20390820503235, 393.2785303592682, 394.3591408729553, 395.23620653152466, 396.1229202747345, 397.21061539649963, 398.0980863571167, 399.2009038925171, 400.09084582328796, 400.97816610336304, 401.88354563713074, 402.7939178943634, 403.91316294670105, 405.0338201522827, 405.95909667015076, 406.8829140663147, 408.03278708457947, 409.1909465789795, 410.34938764572144, 411.30300855636597, 412.2219841480255, 413.1682493686676, 414.3272817134857, 415.29486536979675, 416.25189661979675, 417.242192029953, 418.2236530780792, 419.2278425693512, 420.20685338974, 421.1866135597229, 422.18628549575806, 423.1905610561371, 424.1965022087097, 425.4099974632263, 426.4473731517792, 427.47419142723083, 428.5020925998688, 429.54474329948425, 430.5648584365845, 431.59512186050415, 432.6444957256317, 433.68734335899353, 434.9514834880829, 436.0119893550873, 437.0921359062195, 438.37141942977905, 439.4793417453766, 440.5553512573242, 441.638391494751, 442.71301651000977, 443.7606484889984, 444.8499138355255, 445.906955242157, 446.9736473560333, 448.2631664276123, 449.35534262657166, 450.6643600463867, 451.7735080718994, 452.88786458969116, 454.02642369270325, 455.1352159976959, 456.26882433891296, 457.38967514038086, 458.5427944660187, 459.6806170940399, 461.05473732948303, 462.4692368507385, 463.6257481575012, 464.98427271842957, 466.12405014038086, 467.3002095222473, 468.6563129425049, 469.8273215293884, 471.1904299259186, 472.3828184604645, 473.77507734298706, 474.98277378082275, 476.4182507991791, 477.83399081230164, 479.25034046173096, 480.66764545440674, 481.88703775405884, 483.32665634155273, 484.5651490688324, 486.00618529319763, 487.4562621116638, 488.7013626098633, 489.9261140823364, 491.14500069618225, 493.2321996688843, 494.5062277317047, 495.77761459350586, 497.2556383609772, 498.7606885433197, 500.06691098213196, 501.3513514995575, 502.8534824848175, 504.15090584754944, 505.6636543273926, 506.98424768447876, 508.3061554431915, 509.610422372818, 510.9212510585785, 512.2636876106262, 513.82204413414, 515.3718557357788, 516.737074136734, 518.2937154769897, 519.6247527599335, 520.9895627498627, 522.5908155441284, 524.1691336631775, 525.7543926239014, 527.3603270053864, 528.7805972099304, 530.1968193054199, 531.5912635326385, 533.1889960765839, 534.5816187858582, 536.2026007175446, 537.6152718067169, 539.0977590084076, 540.7226183414459, 542.1220481395721, 543.5343647003174, 544.9389975070953, 546.3412337303162, 547.7787415981293, 549.1963591575623, 550.6050598621368, 552.0186040401459, 553.6495261192322, 555.0836896896362, 556.7850501537323, 558.462694644928, 560.1779072284698, 561.6309621334076, 563.0707976818085, 564.5407590866089, 566.2032699584961, 567.6639289855957, 569.3558588027954, 570.8190577030182, 572.2949397563934, 573.766984462738, 575.2289566993713, 576.7234487533569, 578.2369935512543, 579.7633113861084, 581.2609543800354, 582.787323474884, 584.2983033657074, 585.8132207393646, 587.3319799900055, 588.8701484203339, 590.3878438472748, 591.9030196666718, 593.6539604663849, 595.4025468826294, 597.1395893096924, 598.684508562088, 600.2274060249329, 601.7559370994568, 603.5450429916382, 605.3343744277954, 607.0995283126831, 608.7535207271576, 610.3559763431549, 611.9886453151703, 613.7846167087555, 615.3848061561584, 617.2409067153931, 618.8656992912292, 620.4891543388367, 622.3133592605591, 624.2014274597168, 625.8647367954254, 627.5239741802216, 629.4014942646027, 631.1062579154968, 632.7535409927368, 634.4043598175049, 636.2571125030518, 637.9425766468048, 639.5795505046844, 641.2587592601776, 642.9513120651245, 644.8783185482025, 646.6259920597076, 648.5625722408295, 650.3166482448578, 652.2349772453308, 653.9018788337708, 655.5768268108368, 657.2655262947083, 659.1763205528259, 661.1551268100739, 662.8892042636871, 664.8607544898987, 666.6188313961029, 668.6214656829834, 670.4050114154816, 672.2032294273376, 674.0576498508453, 675.8315184116364, 677.6230657100677, 679.4302287101746, 681.4280540943146, 683.4159116744995, 685.1636734008789, 686.938600063324, 688.7422132492065, 690.5537428855896, 691.464686870575, 692.1828689575195, 692.8850677013397, 693.5827550888062, 694.2790553569794, 694.966591835022, 695.8953537940979, 696.6111404895782, 697.32395195961, 698.048924446106, 698.7974197864532, 699.5067059993744, 700.2285079956055, 700.9705431461334, 701.7027990818024, 702.4572377204895, 703.1842470169067, 703.954351902008, 704.9216766357422, 705.6809704303741, 706.4584770202637, 707.4522190093994, 708.2152233123779, 709.2246136665344, 710.2360401153564, 711.2610895633698, 712.2853305339813, 713.0895235538483, 714.1204483509064, 715.1549866199493, 715.9686377048492, 716.7720413208008, 717.5911648273468, 718.6390790939331, 719.4730575084686, 720.5231368541718, 721.3447744846344, 722.3897793292999, 723.5231442451477, 725.2331948280334, 727.016154050827, 728.4182274341583, 729.2771701812744, 730.1319570541382, 731.2226476669312, 732.1072149276733, 733.2154366970062, 734.0864851474762, 734.9750974178314, 735.8526725769043, 736.9783809185028, 738.0955016613007, 739.2265100479126, 740.1089243888855, 741.0258846282959, 741.9241032600403, 743.0680420398712, 744.244532585144, 745.1702101230621, 746.0981345176697, 747.2638342380524, 748.1796138286591, 749.1140851974487, 750.0738468170166, 751.2459239959717, 752.4520583152771, 753.4394204616547, 754.6566648483276, 755.6376512050629, 756.6307179927826, 757.6211519241333, 758.6132414340973, 759.5859670639038, 760.5710318088531, 761.8012497425079, 762.8146719932556, 764.0478620529175, 765.0574028491974, 766.0820453166962, 767.0876858234406, 768.110100030899, 769.1223406791687, 770.3733065128326, 771.6525964736938, 772.7549026012421, 774.1133015155792, 775.2602920532227, 776.3295385837555, 777.6511747837067, 778.980630159378, 780.0902416706085, 781.175167798996, 782.2748453617096, 783.5883779525757, 784.9424862861633, 786.1000845432281, 787.2481570243835, 788.3972387313843, 789.550940990448, 790.677309513092, 792.0394594669342, 793.1660435199738, 794.3028826713562, 795.4991343021393, 796.8876767158508, 798.3044755458832, 799.4718546867371, 800.6308836936951, 802.2275190353394, 803.8422930240631, 805.0724246501923, 806.5322690010071, 807.7690711021423, 808.9909043312073, 810.2217786312103, 811.4953100681305, 812.7372212409973, 813.9686079025269, 815.432959318161, 816.9058408737183, 818.1418957710266, 819.6087710857391, 821.0900695323944, 822.5935282707214, 823.8253901004791, 825.0650601387024, 826.2767736911774, 827.4890096187592, 828.954847574234, 830.2077457904816, 831.45068359375, 832.7105326652527, 834.194611787796, 835.4705967903137, 836.9795866012573, 838.2710428237915, 839.5227811336517, 840.780513048172, 842.0448169708252, 843.555168390274, 844.8545150756836, 846.1589629650116, 847.4727954864502, 848.7866983413696, 850.3493988513947, 851.9058618545532, 853.2201318740845, 854.5652477741241, 855.8948132991791, 857.4557008743286, 858.7944278717041, 860.1285445690155, 861.4729120731354, 862.8206412792206, 864.4074015617371, 865.7609646320343, 867.1039798259735, 868.462468624115, 869.8398487567902, 871.2438914775848, 872.6199688911438, 874.250524520874, 875.6424853801727, 877.0006048679352, 878.39754986763, 879.7790672779083, 881.1633493900299, 882.5375406742096, 883.93315076828, 885.3530023097992, 886.7477884292603, 888.1825864315033, 889.6914577484131, 891.3900284767151, 892.839558839798, 894.266129732132, 895.7375798225403, 897.2030298709869, 898.7132999897003, 900.4701654911041, 902.0015947818756, 903.5103349685669, 904.9840195178986, 906.4462947845459, 907.9308655261993, 909.4595293998718, 911.0138564109802, 912.5404508113861, 914.0495188236237, 915.5821902751923, 917.1005094051361, 918.6054518222809, 920.430837392807, 922.2437725067139, 923.8361072540283, 925.4303531646729, 927.2418277263641, 928.8749210834503, 930.7062728404999, 932.3419172763824, 934.2204339504242, 935.8275191783905, 937.6460375785828, 939.250785112381, 940.894305229187, 942.55282330513, 944.4473152160645, 946.0852394104004, 947.7636203765869, 949.4436616897583, 951.3113152980804, 952.9697296619415, 954.606077671051, 956.2767400741577, 957.9140491485596, 959.618283033371, 961.5245428085327, 963.2060780525208, 964.9236259460449, 966.8914892673492, 968.6300477981567, 970.3615908622742, 972.3098621368408, 974.0173952579498, 975.9249930381775, 977.8356502056122, 979.5182886123657, 981.4957225322723, 983.5468492507935, 985.5814480781555, 987.3797104358673, 989.407568693161, 991.2472734451294, 993.0739252567291, 994.9354557991028, 996.7823207378387, 998.6259505748749, 1000.4365289211273, 1002.4784958362579, 1004.545102596283, 1006.3781290054321, 1008.1846780776978, 1010.0350351333618, 1011.92405128479, 1013.7983500957489, 1015.6281280517578, 1017.7059783935547, 1019.624892950058, 1021.498117685318, 1023.6381649971008, 1024.3416616916656, 1025.2552840709686, 1025.955050945282, 1026.884395122528, 1027.8182814121246, 1028.5311686992645, 1029.4718053340912, 1030.2094631195068, 1031.1575536727905, 1031.8913695812225, 1032.859093427658, 1033.6042280197144, 1034.5828833580017, 1035.5639040470123, 1036.3375284671783, 1037.3055491447449, 1038.289140701294, 1039.0802731513977, 1039.8494052886963, 1040.6430304050446, 1041.450187444687, 1042.239330291748, 1043.2520403862, 1044.0421121120453, 1044.8404080867767, 1045.6504595279694, 1046.4669694900513, 1047.4991338253021, 1048.5536184310913, 1049.3749659061432, 1050.2007689476013, 1051.025096654892, 1052.0888948440552, 1053.1571667194366, 1054.230155467987, 1055.3010621070862, 1056.3736627101898, 1057.4445524215698, 1058.2982728481293, 1059.1472430229187, 1060.0554809570312, 1060.9352161884308, 1061.8340356349945, 1062.7406578063965, 1063.6431295871735, 1064.564555644989, 1065.4749476909637, 1066.370266199112, 1067.3125076293945, 1068.2448720932007, 1069.1855835914612, 1070.1271846294403, 1071.2894322872162, 1072.4502680301666, 1073.380285024643, 1074.329427242279, 1075.2874748706818, 1076.2367720603943, 1077.2037959098816, 1078.160534620285, 1079.1199436187744, 1080.0942726135254, 1081.0595502853394, 1082.2546963691711, 1083.4581356048584, 1084.666559934616, 1085.6515188217163, 1086.6296482086182, 1087.6265852451324, 1088.8358461856842, 1089.8549802303314, 1090.8832342624664, 1091.9041211605072, 1092.9165637493134, 1094.1614191532135, 1095.1720740795135, 1096.1986088752747, 1097.2140233516693, 1098.2592146396637, 1099.310123205185, 1100.374508857727, 1101.397420167923, 1102.4682366847992, 1103.5118582248688, 1104.5690531730652, 1105.6247148513794, 1106.8998005390167, 1107.9740235805511, 1109.0723168849945, 1110.1620948314667, 1111.248146057129, 1112.5522167682648, 1113.861968755722, 1115.1732580661774, 1116.482558965683, 1117.6006371974945, 1118.9164237976074, 1120.2452411651611, 1121.6013388633728, 1122.9757294654846, 1124.1189119815826, 1125.2554466724396, 1126.6100618839264, 1127.7802164554596, 1128.9139676094055, 1130.2713572978973, 1131.4142136573792, 1132.5660276412964, 1133.7227847576141, 1134.8937866687775, 1136.0548524856567, 1137.2260422706604, 1138.417833328247, 1139.6553208827972, 1141.0893468856812, 1142.2759935855865, 1143.463223695755, 1144.9015221595764, 1146.108249425888, 1147.334114074707, 1148.7635006904602, 1150.2364647388458, 1151.6804451942444, 1153.1357786655426, 1154.3596060276031, 1155.6046814918518, 1156.8509261608124, 1158.325532913208, 1159.6078033447266, 1161.0718030929565, 1162.3434550762177, 1163.8405601978302, 1165.3276569843292, 1166.8108611106873, 1168.0786170959473, 1169.3839149475098, 1170.6937971115112, 1171.989379644394, 1173.5167453289032, 1174.8454298973083, 1176.1389691829681, 1177.6842877864838, 1179.0213506221771, 1180.5881588459015, 1181.9690599441528, 1183.3540546894073, 1184.7680294513702, 1186.1742100715637, 1187.6197476387024, 1189.0349020957947, 1190.6668961048126, 1192.053528547287, 1193.436633348465, 1195.048851966858, 1196.450511932373, 1198.0686945915222, 1199.7257151603699, 1201.133362531662, 1202.590078830719, 1204.057477235794]

# 바운드체크

In [ ]:
from sktime.datatypes._panel._convert import from_2d_array_to_nested
# 전 프레임의 공과 위치 비교 및 이상값제거
for _ in range(3):
    x, y = diff_xy(coords)
    remove_outliers(x, y, coords)

coords = interpolation(coords)

# velocity
Vx = []
Vy = []
V = []
frames = [*range(len(coords))]
print("frames : {}".format(frames))
print(coords)

for i in range(len(coords)-1):
    p1 = coords[i]
    p2 = coords[i+1]
    t1 = check_time[i]
    t2 = check_time[i+1]
    x = (p1[0]-p2[0])/(t1-t2)
    y = (p1[1]-p2[1])/(t1-t2)
    Vx.append(x)
    Vy.append(y)

for i in range(len(Vx)):
    vx = Vx[i]
    vy = Vy[i]
    v = (vx**2+vy**2)**0.5
    V.append(v)

xy = coords[:]

if bounce == 1:
    # Predicting Bounces
    test_df = pandas.DataFrame(
        {'x': [coord[0] for coord in xy[:-1]], 'y': [coord[1] for coord in xy[:-1]], 'V': V})

    # df.shift
    for i in range(20, 0, -1):
        test_df[f'lagX_{i}'] = test_df['x'].shift(i, fill_value=0)
    for i in range(20, 0, -1):
        test_df[f'lagY_{i}'] = test_df['y'].shift(i, fill_value=0)
    for i in range(20, 0, -1):
        test_df[f'lagV_{i}'] = test_df['V'].shift(i, fill_value=0)

    test_df.drop(['x', 'y', 'V'], 1, inplace=True)

    Xs = test_df[['lagX_20', 'lagX_19', 'lagX_18', 'lagX_17', 'lagX_16',
                  'lagX_15', 'lagX_14', 'lagX_13', 'lagX_12', 'lagX_11', 'lagX_10',
                  'lagX_9', 'lagX_8', 'lagX_7', 'lagX_6', 'lagX_5', 'lagX_4', 'lagX_3',
                  'lagX_2', 'lagX_1']]
    Xs = from_2d_array_to_nested(Xs.to_numpy())

    Ys = test_df[['lagY_20', 'lagY_19', 'lagY_18', 'lagY_17',
                  'lagY_16', 'lagY_15', 'lagY_14', 'lagY_13', 'lagY_12', 'lagY_11',
                  'lagY_10', 'lagY_9', 'lagY_8', 'lagY_7', 'lagY_6', 'lagY_5', 'lagY_4',
                  'lagY_3', 'lagY_2', 'lagY_1']]
    Ys = from_2d_array_to_nested(Ys.to_numpy())

    Vs = test_df[['lagV_20', 'lagV_19', 'lagV_18',
                  'lagV_17', 'lagV_16', 'lagV_15', 'lagV_14', 'lagV_13', 'lagV_12',
                  'lagV_11', 'lagV_10', 'lagV_9', 'lagV_8', 'lagV_7', 'lagV_6', 'lagV_5',
                  'lagV_4', 'lagV_3', 'lagV_2', 'lagV_1']]
    Vs = from_2d_array_to_nested(Vs.to_numpy())

    X = pandas.concat([Xs, Ys, Vs], 1)

    # load the pre-trained classifier
    clf = load(open(path + '/Weight_ball/clf.pkl', 'rb'))

    predcted = clf.predict(X)
    idx = list(numpy.where(predcted == 1)[0])
    idx = numpy.array(idx) - 10
    print(idx)
    # 바운드된 프레임이 몇번재 프레임인지 -> 바운드된 좌표랑 프레임 구할 수 있음

    # 트랙킹 완료 하고 돌릴 경우 output_video_path 트랙킹 완료 하지 않고 돌리 경우 input_video_path
    video = cv2.VideoCapture(output_video_path)
    # video = cv2.VideoCapture(input_video_path)

    output_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    output_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))
    num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    print('fps : {}'.format(fps))
    print('frame sizee : {}x{}'.format(output_width, output_height))
    print('num_frames :{}'.format(num_frames))

    output_video = cv2.VideoWriter(path + '/video/final_video.mp4', fourcc, fps, (output_width, output_height))
    bounce_xy = []
    while True:
        ret, frame = video.read()
        if ret:
          if i<len(coords) and coords[i] is not None:
            if i in idx:
                center_coordinates = int(xy[i][0]), int(xy[i][1])
                bounce_xy.append([i,center_coordinates])
                radius = 3
                color = (255, 0, 0)
                thickness = -1
                cv2.circle(frame, center_coordinates, 10, color, thickness)
            i += 1
            output_video.write(frame)
        else:
            break
    video.release()
    output_video.release()

# test

In [ ]:
print(bounce_xy)

# 선수, 탑 뷰에서 보이는 것처럼

In [ ]:

# input 으로 받은 좌표를 직사각형으로 바꿔주는 작업
def order_points(pts):
    # 4, 2 배열을 0으로 초기화 하여 만듦 타입은 float
    rect = np.zeros((4, 2), dtype="float32")
    # pts 는 4x2배열이고 이 배열을 axis 를 1을 기준으로 더하여 4x1배열로 만들어 반환
    # 이를 하는 이유는 바운딩 박스는 두 좌표만 존재하면 구현이 가능하므로
    # 좌 상단은 x,y 의 좌표의 합이 가장 작은 곳이고,
    # 우 하단은 x,y 의 좌표의 합이 가장 큰 곳이기에
    # 두 좌표의 합을 비교하여 좌 상단은 rect[0],
    # 우 하단은 rect[2] 에
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # diff 를 x에서 y를 뺏을 때의 값을 구함
    # 우 상단은 diff 의 값이 가장 작은 곳
    # 좌 하단은 diff 의 값이 가장 큰 곳이기에
    # rect[1] 에는 우 상단, rect[3] 은 좌 하단
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect


# 코트의 좌표를 생성하는 모듈
def transition_matrix(image_pts, bev_pts):
    rect_image = order_points(image_pts)
    rect_bev = order_points(bev_pts)
    # getPerspectiveTransform 해당 함수는 직사각형이 아닌 좌표를 경기장 규격에 맞게 투시 변환을 해주기 위해 사용
    M = cv2.getPerspectiveTransform(rect_image, rect_bev)
    return M


# width 기준으로 코드 비율 맞춰 코드 변환
def court_coord(width, pad):
    height = int(width * (1 - pad) * 2 * (1 + pad))
    x_1, y_1 = int(width * pad), int(height * pad)
    x_2, y_2 = int(width * (1 - pad)), int(height * pad)
    x_3, y_3 = int(width * (1 - pad)), int(height * (1 - pad))
    x_4, y_4 = int(width * pad), int(height * (1 - pad))
    return [(x_1, y_1), (x_2, y_2), (x_3, y_3), (x_4, y_4)]


def player_coord(coord_image, M):
    X = list(coord_image)
    X.append(1)
    X = np.array(X)
    # X의 행렬 곱의 결과
    Y = M.dot(X)
    Y = tuple(map(lambda x: round(x / Y[2]), Y))
    return (Y[0], Y[1])


class top_view_court:
    def __init__(self, width, pad):
        line_color = (255, 255, 255)
        court_color_1 = (192, 158, 128)
        court_color_2 = (153, 112, 80)
        

        height = int(width * (1 - pad) * 2 * (1 + pad))
        self.court = np.zeros((height, width, 3), np.uint8)

        cv2.rectangle(self.court, (0, 0), (width, height), court_color_1, -1)

        # 외곽선
        x_1, y_1 = int(width * pad), int(height * pad)
        x_2, y_2 = int(width * (1 - pad)), int(height * pad)
        x_3, y_3 = int(width * (1 - pad)), int(height * (1 - pad))
        x_4, y_4 = int(width * pad), int(height * (1 - pad))

        cv2.rectangle(self.court, (x_1, y_1), (x_3, y_3), court_color_2, -1)
        cv2.line(self.court, (x_1, y_1), (x_2, y_2), line_color, 2)
        cv2.line(self.court, (x_2, y_2), (x_3, y_3), line_color, 2)
        cv2.line(self.court, (x_3, y_3), (x_4, y_4), line_color, 2)
        cv2.line(self.court, (x_4, y_4), (x_1, y_1), line_color, 2)

        # 실제 경기장 비율
        x_ratio = (x_2 - x_1) / 10.97
        y_ratio = (y_3 - y_2) / 23.78

        # 복식 라인
        xc_1, yc_1 = int(x_1 + x_ratio * 1.372), y_1
        xc_2, yc_2 = int(x_2 - x_ratio * 1.372), y_2
        xc_3, yc_3 = int(x_3 - x_ratio * 1.372), y_3
        xc_4, yc_4 = int(x_4 + x_ratio * 1.372), y_4

        cv2.line(self.court, (xc_1, yc_1), (xc_4, yc_4), line_color, 2)
        cv2.line(self.court, (xc_2, yc_2), (xc_3, yc_3), line_color, 2)

        # 서브라인
        xs_1, ys_1 = xc_1, int(y_1 + 5.50 * y_ratio)
        xs_2, ys_2 = xc_2, int(y_2 + 5.50 * y_ratio)
        xs_3, ys_3 = xc_3, int(y_3 - 5.50 * y_ratio)
        xs_4, ys_4 = xc_4, int(y_4 - 5.50 * y_ratio)

        cv2.line(self.court, (xs_1, ys_1), (xs_2, ys_2), line_color, 2)
        cv2.line(self.court, (xs_3, ys_3), (xs_4, ys_4), line_color, 2)

        # 네트
        xnet_1, ynet_1 = x_1, int((y_4 - y_1) / 2 + y_1)
        xnet_2, ynet_2 = x_2, int((y_4 - y_1) / 2 + y_1)

        cv2.line(self.court, (xnet_1, ynet_1), (xnet_2, ynet_2), line_color, 2)

        # 가운데 서브 라인
        xv_1, yv_1 = int((x_2 - x_1) / 2 + x_1), ys_1
        xv_2, yv_2 = int((x_2 - x_1) / 2 + x_1), ys_3
        cv2.line(self.court, (xv_1, yv_1,), (xv_2, yv_2), line_color, 2)

        # 중앙 마크 표시
        xm = int((x_2 - x_1) / 2 + x_1)
        ym_1 = y_1
        ym_2 = int(y_1 + 10)
        ym_3 = int(y_4 - 10)
        ym_4 = y_4

        cv2.line(self.court, (xm, ym_1), (xm, ym_2), line_color, 2)
        cv2.line(self.court, (xm, ym_3), (xm, ym_4), line_color, 2)

    # 선수 그리기
    def add_player(self, coord_bev, n_player, color_player_1, color_player_2):
        x, y = coord_bev
        if n_player == 0:
            cv2.circle(self.court, (x, y), radius=7, color=color_player_1, thickness=-1)
            cv2.circle(self.court, (x, y), radius=7, color=(255, 255, 255), thickness=2)
        elif n_player == 1:
            cv2.circle(self.court, (x, y), radius=7, color=color_player_2, thickness=-1)
            cv2.circle(self.court, (x, y), radius=7, color=(255, 255, 255), thickness=2)

    def add_ball(self, coord_bev, color, radius):
      x, y = coord_bev
      # if(i in idx):
      #   cv2.circle(self.court, (x,y), radius = 10, color=(255,0,0), thickness = -1)
      #   print(coord_bev)
      #   self.bounce_xy_test.append([x, y])
      # else:
      cv2.circle(self.court, (x,y), radius = radius, color=color, thickness = -1)

    # 선수 이동 경로 그리기
    def add_path_player(self, coord_bev, color_path=(255, 255, 255)):
        x, y = coord_bev
        cv2.circle(self.court, (x, y), radius=1, color=color_path, thickness=-1)

    def add_path_ball(self, coord_bev, color_path=(0,0,0)):
        x, y = coord_bev
        cv2.circle(self.court, (x, y), radius=1, color=color_path, thickness=-1)


# 영상출력 크기
output_width = 1000
pad = 0.22
output_height = int(output_width * (1 - pad) * 2 * (1 + pad))

# 코트의 상하좌우 좌표 -> 영상마다 바꿔야 함
image_pts = np.array([(574, 307), (1338, 307), (1566, 871), (363, 871)]).reshape(4, 2)
bev_pts = np.array(court_coord(output_width, pad)).reshape(4, 2)
# 촬영한 영상을 탑 뷰에서 보여주기 위해 두 좌표의 차이를 반환
M = transition_matrix(image_pts, bev_pts)

# 이미 트래킹을 끝낸 후 가공된 csv 파일 불러오기
position_df = pd.read_csv('/content/drive/MyDrive/analysis_application/tracking_players.csv')
position_df['cp_0'] = list(zip(position_df.x_0, position_df.y_0))
position_df['cp_1'] = list(zip(position_df.x_1, position_df.y_1))
position_df['coord_bev_0'] = position_df['cp_0'].apply(lambda x: player_coord(x, M))
position_df['coord_bev_1'] = position_df['cp_1'].apply(lambda x: player_coord(x, M))
position_0 = list(position_df['coord_bev_0'])  # 선수 1
position_1 = list(position_df['coord_bev_1'])  # 선수 2

# 이미 트래킹을 끝낸 공 위치 csv 파일 불러오기
position_ball_df = pd.read_csv('/content/drive/MyDrive/analysis_application/tracking_ball.csv')
position_ball_df['cp'] = list(zip(position_ball_df.x_0, position_ball_df.y_0))
position_ball_df['coord_bev'] = position_ball_df['cp'].apply(lambda x: player_coord(x, M))
position_ball = list(position_ball_df['coord_bev'])  # 선수 1

# top view 영상 저장
# path = '/content/drive/MyDrive/analysis_application'
output_video_path = path + '/video/output_top_view.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 60

output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (output_width, output_height))

court_base = top_view_court(output_width, pad)

bounce_xy_test = []
# top view 영상 저장을 위해
i = 0
while True:
    if len(position_0) == i:
        print("break")
        break
    if len(position_ball) == i:
      print("ball break")
      break
    # 선수 경로같은 경우는 계속해서 축적되어야 하고, 선수같은 경우 해당 프레임에서만 그려줘야 하므로
    court = deepcopy(court_base)
    # 색상 변경 하자
    court.add_player(position_0[i], 0, (255, 0, 0), (0, 0, 0))
    court.add_player(position_1[i], 0, (38, 19, 15), (0, 0, 0))
    if(i in idx): # 바운드볼
      court.add_ball(position_ball[i], (255,0,0,), 10)
      bounce_xy_test.append([i, position_ball[i]])
    else:
      court.add_ball(position_ball[i], (0,255,0,), 4)

    court_base.add_path_player(position_0[i])
    court_base.add_path_player(position_1[i])
    # court_base.add_path_ball(position_ball[i])

    output_video.write(court.court)
    i += 1

output_video.release()
print(bounce_xy_test)

# 바운드 test

video_cut   
bounce_xy_frame = [[22, (912, 596)], [23, (901, 634)], [24, (896, 647)], [25, (888, 651)], [26, (880, 655)], [27, (873, 660)], [28, (872, 661)], [29, (865, 665)], [179, (739, 655)], [180, (733, 676)], [253, (688, 356)], [254, (689, 355)], [255, (689, 354)], [256, (689, 353)], [257, (689, 352)], [258, (688, 340)], [260, (690, 318)], [261, (690, 310)], [262, (691, 311)], [427, (1204, 352)], [428, (1202, 347)], [429, (1203, 334)], [430, (1202, 338)], [431, (1201, 325)], [587, (822, 420)], [588, (820, 421)], [589, (816, 424)], [590, (813, 428)], [591, (810, 420)], [592, (811, 420)], [593, (807, 399)], [594, (806, 374)], [595, (803, 355)], [596, (801, 338)], [597, (799, 319)], [731, (974, 425)], [743, (976, 430)], [744, (975, 424)], [746, (975, 413)], [803, (896, 419)]]


In [ ]:
# 변환된 바운드 좌표와 바운드된 프레임 수
bounce_xy_test = [[25, (476, 1043)], [26, (468, 1112)], [27, (464, 1165)], [28, (459, 1173)], [29, (454, 1181)], [30, (451, 1187)], [31, (451, 1187)], [32, (446, 1197)], [179, (382, 1116)], [180, (382, 1131)], [253, (312, 565)], [254, (311, 560)], [255, (311, 557)], [256, (311, 549)], [257, (310, 549)], [258, (311, 546)], [260, (311, 541)], [261, (309, 510)], [262, (305, 482)], [427, (677, 541)], [428, (676, 543)], [429, (675, 541)], [430, (673, 546)], [431, (672, 541)], [587, (414, 710)], [588, (412, 710)], [589, (412, 708)], [590, (410, 717)], [591, (408, 722)], [592, (407, 729)], [593, (403, 731)], [594, (403, 731)], [595, (402, 727)], [596, (398, 683)], [597, (395, 615)], [731, (511, 703)], [743, (541, 984)], [744, (531, 909)], [746, (510, 743)], [803, (467, 710)], [804, (464, 712)], [805, (463, 710)]]
output_width = 1000
pad = 0.22
output_height = int(output_width * (1 - pad) * 2 * (1 + pad))

# 변환하지 않은 바운드 좌표, 프레임 수
bounce_xy = [[22, (912, 596)], [23, (901, 634)], [24, (896, 647)], [25, (888, 651)], [26, (880, 655)], [27, (873, 660)], [28, (872, 661)], [29, (865, 665)], [179, (739, 655)], [180, (733, 676)], [253, (688, 356)], [254, (689, 355)], [255, (689, 354)], [256, (689, 353)], [257, (689, 352)], [258, (688, 340)], [260, (690, 318)], [261, (690, 310)], [262, (691, 311)], [427, (1204, 352)], [428, (1202, 347)], [429, (1203, 334)], [430, (1202, 338)], [431, (1201, 325)], [587, (822, 420)], [588, (820, 421)], [589, (816, 424)], [590, (813, 428)], [591, (810, 420)], [592, (811, 420)], [593, (807, 399)], [594, (806, 374)], [595, (803, 355)], [596, (801, 338)], [597, (799, 319)], [731, (974, 425)], [743, (976, 430)], [744, (975, 424)], [746, (975, 413)], [803, (896, 419)]]

# 1. 연속된 값들 제거
stack = []
frame_test = []
for i in bounce_xy_test:
  frame_test.append(i[0])
  if not stack:
    stack.append(i)
  elif stack[-1][0]+1==i[0]:
    stack.pop()
    stack.append(i)
  else:
    stack.append(i)

print(stack)
# print(frame_test)
# print(output_height)

# 2. 위, 아래 중 어디에 바운드 탑뷰로 변환 후 기준
for i in stack:
  if(i[1][1]>=output_height/2):
    print("아래",i)
  else:
    print("위",i)
# # 2. 위, 아래 중 어디에 바운드 영상이 854x480 기준
# for i in stack:
#   if(i[1][1]>=240):
#     print("아래",i)
#   else:
#     print("위",i)